## Name: Transfer learning using LMs on KGs
### Date: 12/3/2024
### Status: Not sure what we wanted to see
### Idea: 
The idea was to embed the nodes and relations in a way that would enable transfer learning between KGs.

### Results:
Currently in this notebook:
1. We load FB15k-237.
2. We map entities back to original names.
3. We encode them with a sentence transformer.
4. We perform gcn pass (needs work).



In [1]:
import pandas as pd


df = pd.read_csv("../Prime_Adj/data/FB15k-237/train.txt", sep='\t', header=None)
df.columns = ['head', 'rel', 'tail']
df

,head,rel,tail
0,/m/027rn,/location/country/form_of_government,/m/06cx9
1,/m/017dcd,/tv/tv_program/regular_cast./tv/regular_tv_app...,/m/06v8s0
2,/m/07s9rl0,/media_common/netflix_genre/titles,/m/0170z3
3,/m/01sl1q,/award/award_winner/awards_won./award/award_ho...,/m/044mz_
4,/m/0cnk2q,/soccer/football_team/current_roster./sports/s...,/m/02nzb8
...,...,...,...
272110,/m/016mhd,/film/film/release_date_s./film/film_regional_...,/m/029j_
272111,/m/01g888,/music/genre/artists,/m/01vv126
272112,/m/0djvzd,/soccer/football_player/current_team./sports/s...,/m/02b16p
272113,/m/0gmblvq,/award/award_winning_work/awards_won./award/aw...,/m/07kjk7c


In [2]:
df_map = pd.read_csv("fb_map.txt", sep='	', header=None)
df_map.columns = ['ent', 'descr']
map_dict = dict(zip(df_map.ent, df_map.descr))
df['head_d'] = df['head'].map(map_dict)
df['tail_d'] = df['tail'].map(map_dict)
df

,head,rel,tail,head_d,tail_d
0,/m/027rn,/location/country/form_of_government,/m/06cx9,languages of dominican republic (Q786),republic (government) (Q7270)
1,/m/017dcd,/tv/tv_program/regular_cast./tv/regular_tv_app...,/m/06v8s0,rangers in reverse (Q513809),elyse floyd (Q2642437)
2,/m/07s9rl0,/media_common/netflix_genre/titles,/m/0170z3,drama movie (Q130232),danny vinyard (Q208572)
3,/m/01sl1q,/award/award_winner/awards_won./award/award_ho...,/m/044mz_,michelle rodríguez (Q184219),naveen andrews (Q208374)
4,/m/0cnk2q,/soccer/football_team/current_roster./sports/s...,/m/02nzb8,australia national soccer team (Q268208),right winger (football) (Q193592)
...,...,...,...,...,...
272110,/m/016mhd,/film/film/release_date_s./film/film_regional_...,/m/029j_,billy elliott (film) (Q458629),dual layer (Q5294)
272111,/m/01g888,/music/genre/artists,/m/01vv126,skunk rock (Q460674),benji combs (Q792976)
272112,/m/0djvzd,/soccer/football_player/current_team./sports/s...,/m/02b16p,delroy facey/comments (Q3021900),lincoln city football club (Q18519)
272113,/m/0gmblvq,/award/award_winning_work/awards_won./award/aw...,/m/07kjk7c,game change (film) (Q655980),primetime emmy award for outstanding writing f...


In [3]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

rel2emb = {}
for rel in df['rel'].unique():
    rel2emb[rel] = model.encode(rel)

In [4]:
import tqdm
node2emb = {}
ents = set(df['head_d'].unique().tolist() + df['tail_d'].unique().tolist())
for node in tqdm.notebook.tqdm(ents, total=len(ents)):
    node2emb[node] = model.encode(node)

  0%|          | 0/14505 [00:00<?, ?it/s]

In [ ]:
len(node2emb), len(rel2emb)

In [26]:
## Network

In [36]:
rel2id = {r:index for index, r in enumerate(df['rel'].unique().tolist())}
node2id = {r:index for index, r in enumerate(ents)}


In [59]:
i,v = [], []
for _,row in df.iterrows():
    i.append([node2id[row['head_d']], node2id[row['tail_d']]])
    v.append(rel2id[row['rel']] + 1)
    #break
adj = torch.sparse_coo_tensor(torch.Tensor(i).T, v, size=(len(ents), len(ents)))

In [117]:
import torch


tensor([[[0.4153, 0.7691, 0.1945,  ..., 0.9695, 0.6074, 0.8619],
         [0.5083, 0.9575, 0.3566,  ..., 0.7266, 0.1044, 0.8752],
         [0.3493, 0.8221, 0.5775,  ..., 0.2128, 0.0955, 0.1314],
         ...,
         [0.0848, 0.3229, 0.6577,  ..., 0.4853, 0.5514, 0.0212],
         [0.4624, 0.2532, 0.7025,  ..., 0.0465, 0.5206, 0.4878],
         [0.0017, 0.6629, 0.3637,  ..., 0.8447, 0.4028, 0.7554]],

        [[0.3045, 0.3092, 0.2874,  ..., 0.8428, 0.4219, 0.1997],
         [0.1417, 0.0717, 0.9802,  ..., 0.6823, 0.1697, 0.9144],
         [0.0364, 0.2827, 0.9965,  ..., 0.9086, 0.8758, 0.6149],
         ...,
         [0.5413, 0.3850, 0.9403,  ..., 0.1194, 0.7747, 0.4802],
         [0.8413, 0.4859, 0.3991,  ..., 0.4940, 0.9605, 0.5224],
         [0.7556, 0.4568, 0.9928,  ..., 0.1343, 0.2898, 0.2799]],

        [[0.0015, 0.4084, 0.5068,  ..., 0.3346, 0.8359, 0.1956],
         [0.3581, 0.0582, 0.9918,  ..., 0.2411, 0.5972, 0.3376],
         [0.5951, 0.2775, 0.0790,  ..., 0.3600, 0.9581, 0.

In [118]:
import torch
rel_init_tensor = torch.Tensor(list(rel2emb.values()))
node_init_tensor = torch.Tensor(list(node2emb.values()))

In [126]:
def gcn_pass(adj, node_feats, W):
    # NxN @ NxF = NxF
    #print(adj, node_feats)
    community = adj @ node_feats
    # NxF @ FxH = NxH
    #print(community.shape, W.shape)
    proj = community @ W
    proj = torch.sigmoid(proj)
    return proj
    

num_epochs = 100
all_edges = adj.coalesce().indices()
all_values = adj.coalesce().values()

for epoch in range(num_epochs):
    w = torch.einsum('rb,bbh->rbh', rel_init_tensor, theta)
    cur_proj = torch.zeros(len(ents), hidden_size)
    for rel_index, current_W in enumerate(w):
        cur_mask = (all_values - 1) == rel_index
        cur_edges = all_edges[:,cur_mask].long()
        cur_adj = torch.sparse_coo_tensor(cur_edges, torch.Tensor([1]*len(cur_edges.T)).float(), size=(len(ents), len(ents)))
        cur_proj += gcn_pass(cur_adj.to_sparse_csr(), node_init_tensor, current_W)
    cur_proj /= w.shape[0]
    for _, row in df.iterrows():
        break
    break